In [1]:
%matplotlib inline

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import argparse
import datetime
import importlib
import os
import tensorflow as tf
import yaml
import math
from train_utils import *
from tensorflow.python.framework import graph_util
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [31]:
file_address = open('config/scannet/test/joined_bal.config')
config = yaml.load(file_address)

/nethome/shickson3/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [32]:
module = importlib.import_module('models.' + config['model'])
model_func = getattr(module, config['model'])
dataset_name = config['dataset_name']
helper = get_dataset(config)
#helper.Setup(config)
modality_infos, num_label_classes = extract_modalities(config)
data_list, iterator = helper.get_test_data(config, num_label_classes)
resnet_name = 'resnet_v2_50'

with tf.variable_scope(resnet_name):
    model = model_func(modality_infos=modality_infos, training=False)
    images_pl, depths_pl, normals_pl, labels_pl, update_ops = setup_model_flops(model, config, train=False)

config1 = tf.ConfigProto()
#config1.gpu_options.allow_growth = True
sess = tf.Session(config=config1)
sess.run(tf.global_variables_initializer())
import_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
print 'total_variables_loaded:', len(import_variables)
#output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['resnet_v2_50/labels/batchnorm_1/add_1','resnet_v2_50/normals/batchnorm_1/add_1',])
output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['resnet_v2_50/labels/conv5/batchnorm_1/add_1','resnet_v2_50/normals/conv5/batchnorm_1/add_1',])
#output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['resnet_v2_50/conv5/batchnorm_1/add_1'])
#output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['resnet_v2_50/decoder/batchnorm_1/add_1'])

# Below computes the computations up to the aux loss.
#output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['resnet_v2_50/resize_1/ResizeBilinear',])
#output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['resnet_v2_50/decoder/resize_1/ResizeBilinear',])
#output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['resnet_v2_50/labels/resize_1/ResizeBilinear','resnet_v2_50/normals/resize_1/ResizeBilinear',])
with tf.gfile.GFile('/dev/shm/graph.pb', "wb") as f:
    f.write(output_graph_def.SerializeToString())

Network_Construction
total_variables_loaded: 438


In [33]:
def load_pb(pb):
    with tf.gfile.GFile(pb, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

g2 = load_pb('/dev/shm/graph.pb')
with g2.as_default():
    run_metadata = tf.RunMetadata()
    flops = tf.profiler.profile(g2, options=tf.profiler.ProfileOptionBuilder.float_operation())
print('FLOP = ', flops.total_float_ops)

# run_metadata = tf.RunMetadata()
# flops = tf.profiler.profile(tf.get_default_graph(), options=tf.profiler.ProfileOptionBuilder.float_operation())
# print('FLOP = ', flops.total_float_ops)

('FLOP = ', 216323076702L)


In [34]:
tf.reset_default_graph()

In [29]:
for n in tf.get_default_graph().as_graph_def().node:
    print n.name

tensors/normalize_tensors/component_0
tensors/normalize_tensors/component_1
tensors/normalize_tensors/component_2
TensorSliceDataset
MapDataset
num_parallel_calls
ParallelMapDataset
batch_size
drop_remainder
BatchDatasetV2
buffer_size
PrefetchDataset
OneShotIterator
IteratorToStringHandle
IteratorGetNext
resnet_v2_50/ones/shape_as_tensor
resnet_v2_50/ones/Const
resnet_v2_50/ones
resnet_v2_50/zeros/shape_as_tensor
resnet_v2_50/zeros/Const
resnet_v2_50/zeros
resnet_v2_50/concat/axis
resnet_v2_50/concat
resnet_v2_50/ones_1/shape_as_tensor
resnet_v2_50/ones_1/Const
resnet_v2_50/ones_1
resnet_v2_50/zeros_1/shape_as_tensor
resnet_v2_50/zeros_1/Const
resnet_v2_50/zeros_1
resnet_v2_50/concat_1/axis
resnet_v2_50/concat_1
resnet_v2_50/ones_2/shape_as_tensor
resnet_v2_50/ones_2/Const
resnet_v2_50/ones_2
resnet_v2_50/ones_3/shape_as_tensor
resnet_v2_50/ones_3/Const
resnet_v2_50/ones_3
resnet_v2_50/ArgMax/dimension
resnet_v2_50/ArgMax
resnet_v2_50/ones_4/shape_as_tensor
resnet_v2_50/ones_4/Const
re

resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/weights//Regularizer/l2_regularizer/scale
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/weights//Regularizer/l2_regularizer/L2Loss
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/weights//Regularizer/l2_regularizer
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/Conv2D
resnet_v2_50/block3/unit_5/bottleneck_v1/Add
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma/Initializer/Const
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma/Assign
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma/read
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma/Regularizer/l2_regularizer/scale
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma/Regularizer/l2_regularizer/L2Loss
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma/Regularizer/l2_regularizer
resnet_v2_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/beta/Initializer/

resnet_v2_50/conv248/BatchNorm/gamma/read
resnet_v2_50/conv248/BatchNorm/gamma/Regularizer/l2_regularizer/scale
resnet_v2_50/conv248/BatchNorm/gamma/Regularizer/l2_regularizer/L2Loss
resnet_v2_50/conv248/BatchNorm/gamma/Regularizer/l2_regularizer
resnet_v2_50/conv248/BatchNorm/beta/Initializer/Const
resnet_v2_50/conv248/BatchNorm/beta
resnet_v2_50/conv248/BatchNorm/beta/Assign
resnet_v2_50/conv248/BatchNorm/beta/read
resnet_v2_50/conv248/BatchNorm/beta/Regularizer/l2_regularizer/scale
resnet_v2_50/conv248/BatchNorm/beta/Regularizer/l2_regularizer/L2Loss
resnet_v2_50/conv248/BatchNorm/beta/Regularizer/l2_regularizer
resnet_v2_50/conv248/BatchNorm/moving_mean/Initializer/Const
resnet_v2_50/conv248/BatchNorm/moving_mean
resnet_v2_50/conv248/BatchNorm/moving_mean/Assign
resnet_v2_50/conv248/BatchNorm/moving_mean/read
resnet_v2_50/conv248/BatchNorm/moving_variance/Initializer/Const
resnet_v2_50/conv248/BatchNorm/moving_variance
resnet_v2_50/conv248/BatchNorm/moving_variance/Assign
resnet_v2